# Export Data for Analysis

This notebook just lets me reshape the csvs into ones that are more convenient for non-resim analysis tasks. Currently loading the `*-out.csv` files and saving a subset of the columns to single-season csvs, for the purposes of examining "real" defensive contributions.

In [1]:
import glob

import pandas as pd
import numpy as np
from csv_dtypes import column_dtypes

In [2]:
all_files = glob.glob("../roll_data/*-out.csv")

df = pd.concat((pd.read_csv(f, dtype=column_dtypes) for f in all_files), ignore_index=True)

# seasons = [11, 12]
# seasons = [11, 12, 13, 14]
# seasons = [14]
# seasons = [13]
# df = df[df['season'].isin(seasons)]
len(df)


220860

In [3]:
def get_pitcher_mul(row):
    pm = str(row["pitcher_mods"])
    ptm = str(row["pitching_team_mods"])
    mul = 1
    if "OVERPERFORMING" in pm:
        mul += 0.2
    if "OVERPERFORMING" in ptm:
        mul += 0.2
    if "UNDERPERFORMING" in pm:
        mul -= 0.2
    if "UNDERPERFORMING" in ptm:
        mul -= 0.2
    if "TRAVELING" in ptm:
        # pass # sometimes traveling doesn't do anything on pitchers, but not always
        if not row["top_of_inning"]:
            mul += 0.05
    if "HIGH_PRESSURE" in ptm:
        if row["weather"] == "Weather.FLOODING" and row["baserunner_count"] > 0:
            mul += 0.25
    if "GROWTH" in ptm:
        # pass # sometimes growth doesn't do anything on pitchers, but not always
        mul += min(0.05, 0.05 * (row["day"] / 99))
    if "SINKING_SHIP" in ptm:
        mul += (14 - row["pitching_team_roster_size"]) * 0.01
    if "AFFINITY_FOR_CROWS" in ptm and row["weather"] == "Weather.BIRDS":
        mul += 0.5
    # if row["pitcher_name"] == "Sutton Dreamy" and row["weather"] == "Weather.ECLIPSE":
        # NVGs allow the player to play 50% better in a solar eclipse.
        # This might affect fielding and baserunning as well?
        # this will 100% break if Dreamy gets scattered. And when they lose the old NVGs.
        # Did anyone with this mod *ever* pitch???
        # mul += 0.50
    return mul


def get_batter_mul(row):
    bm = str(row["batter_mods"])
    btm = str(row["batting_team_mods"])
    ptm = str(row["pitching_team_mods"])
    mul = 1
    if "OVERPERFORMING" in bm:
        mul += 0.2
    if "OVERPERFORMING" in btm:
        mul += 0.2
    if "UNDERPERFORMING" in bm:
        mul -= 0.2
    if "UNDERPERFORMING" in btm:
        mul -= 0.2
    if "TRAVELING" in btm:
        # pass
        if row["top_of_inning"]:
            mul += 0.05
    if "GROWTH" in btm:
        # pass
        mul += min(0.05, 0.05 * (row["day"] / 99))
    if "HIGH_PRESSURE" in btm:
        if row["weather"] == "Weather.FLOODING" and row["baserunner_count"] > 0:
            mul += 0.25
    if "SINKING_SHIP" in btm:
        mul += (14 - row["batting_team_roster_size"]) * 0.01
    if "AFFINITY_FOR_CROWS" in btm and row["weather"] == "Weather.BIRDS":
        mul += 0.5
    if "CHUNKY" in bm and row["weather"] == "Weather.PEANUTS":
        # I have handling for these but it's clumsy and possibly easier to do "manually"
        pass
    if "SMOOTH" in bm and row["weather"] == "Weather.PEANUTS":
        # I have handling for these but it's clumsy and possibly easier to do "manually"
        pass
    if "ON_FIRE" in bm:
        # todo: handle properly, which requires figuring out how it works, which sucks
        mul += 2.0
    # if row["batter_name"] == "Sutton Dreamy" and row["weather"] == "Weather.ECLIPSE":
        # NVGs allow the player to play 50% better in a solar eclipse.
        # This might affect fielding and baserunning as well?
        # this will 100% break if Dreamy gets scattered. And when they lose the old NVGs.
        # mul += 0.50
    return mul

# patch the fielder multiplier, which missed Shelled when done in resim.py (last time I ran it anyway)
def get_fielder_mul(row):
    fm = str(row["fielder_mods"])
    ptm = str(row["pitching_team_mods"])
    mul = 1
    if "OVERPERFORMING" in fm:
        mul += 0.2
    if "OVERPERFORMING" in ptm:
        mul += 0.2
    if "UNDERPERFORMING" in fm:
        mul -= 0.2
    if "UNDERPERFORMING" in ptm:
        mul -= 0.2
    if "TRAVELING" in ptm:
        # pass
        if not row["top_of_inning"]:
            mul += 0.05
    if "HIGH_PRESSURE" in ptm:
        if row["weather"] == "Weather.FLOODING" and row["baserunner_count"] > 0:
            mul += 0.25
    if "GROWTH" in ptm:
        mul += min(0.05, 0.05 * (row["day"] / 99))
    if "SINKING_SHIP" in ptm:
        mul += (14 - row["pitching_team_roster_size"]) * 0.01
    # if "AFFINITY_FOR_CROWS" in ptm and row["weather"] == "Weather.BIRDS":
        # mul += 0.5  # doesn't apply to fielders I guess?
    if "SHELLED" in fm:
        # is it this, or is it "mul = 0", I wonder
        mul -= 1.0
    # if row["fielder_name"] == "Sutton Dreamy" and row["weather"] == "Weather.ECLIPSE":
        # NVGs allow the player to play 50% better in a solar eclipse.
        # This might affect fielding and baserunning as well?
        # mul += 0.50
    return mul #if mul == 0 else 1.0

# df["batter_multiplier"] = df.apply(get_batter_mul, axis=1)
# df["pitcher_multiplier"] = df.apply(get_pitcher_mul, axis=1)
# df["fielder_multiplier"] = df.apply(get_fielder_mul, axis=1)

In [4]:
# df["batter_mul"] = df.apply(get_batter_mul, axis=1)
# df["pitcher_mul"] = df.apply(get_pitcher_mul, axis=1)
# df["fielder_mul"] = df.apply(get_fielder_mul, axis=1)

In [5]:
# zero out scattered vibes
df.loc[df["pitcher_mods"].astype(str).str.contains("SCATTERED"), "pitcher_vibes"] = 0
df.loc[df["batter_mods"].astype(str).str.contains("SCATTERED"), "batter_vibes"] = 0
df.loc[df["fielder_mods"].astype(str).str.contains("SCATTERED"), "fielder_vibes"] = 0

for attr in [
    "batter_buoyancy",
    "batter_divinity",
    "batter_martyrdom",
    "batter_moxie",
    "batter_musclitude",
    "batter_patheticism",
    "batter_thwackability",
    "batter_tragicness",
    "batter_coldness",
    "batter_overpowerment",
    "batter_ruthlessness",
    "batter_shakespearianism",
    "batter_suppression",
    "batter_unthwackability",
    "batter_base_thirst",
    "batter_continuation",
    "batter_ground_friction",
    "batter_indulgence",
    "batter_laserlikeness",
    "batter_anticapitalism",
    "batter_chasiness",
    "batter_omniscience",
    "batter_tenaciousness",
    "batter_watchfulness",
    "batter_pressurization",
    "batter_cinnamon",
]:
    df[attr + "_mul_vibe"] = df[attr] * df["batter_multiplier"] * (1 + 0.2 * df["batter_vibes"])

for attr in [
    "pitcher_buoyancy",
    "pitcher_divinity",
    "pitcher_martyrdom",
    "pitcher_moxie",
    "pitcher_musclitude",
    "pitcher_patheticism",
    "pitcher_thwackability",
    "pitcher_tragicness",
    "pitcher_ruthlessness",
    "pitcher_overpowerment",
    "pitcher_unthwackability",
    "pitcher_shakespearianism",
    "pitcher_suppression",
    "pitcher_coldness",
    "pitcher_base_thirst",
    "pitcher_continuation",
    "pitcher_ground_friction",
    "pitcher_indulgence",
    "pitcher_laserlikeness",
    "pitcher_anticapitalism",
    "pitcher_chasiness",
    "pitcher_omniscience",
    "pitcher_tenaciousness",
    "pitcher_watchfulness",
    "pitcher_pressurization",
    "pitcher_cinnamon",
]:
    df[attr + "_mul_vibe"] = df[attr] * df["pitcher_multiplier"] * (1 + 0.2 * df["pitcher_vibes"])

if "fielder_vibes" in df:
    for attr in [
        "fielder_anticapitalism",
        "fielder_chasiness",
        "fielder_omniscience",
        "fielder_tenaciousness",
        "fielder_watchfulness",
    ]:
        df[attr + "_mul_vibe"] = df[attr] * df["fielder_multiplier"] * (1 + 0.2 * df["fielder_vibes"])

df["grand_center"] = df["ballpark_grandiosity"] - 0.5
df["fort_center"]  = df["ballpark_fortification"] - 0.5
df["obt_center"]   = df["ballpark_obtuseness"] - 0.5
df["omi_center"]   = df["ballpark_ominousness"] - 0.5
df["fwd_center"]   = df["ballpark_forwardness"] - 0.5
df["visc_center"]  = df["ballpark_viscosity"] - 0.5
df["incon_center"] = df["ballpark_inconvenience"] - 0.5
df["elong_center"] = df["ballpark_elongation"] - 0.5

df = df.copy()

In [6]:
out_cols = [
    "event_type",
    "roll",
    "passed",
    "batter_id",
    "batter_name",
    "batter_thwackability",
    "batter_thwackability_mul_vibe",
    "batter_multiplier",
    "batter_vibes",
    "batter_mods",
    "batting_team_mods",
    "pitcher_id",
    "pitcher_name",
    "pitcher_unthwackability",
    "pitcher_unthwackability_mul_vibe",
    "pitcher_multiplier",
    "pitcher_vibes",
    "pitcher_mods",
    "pitching_team_mods",
    "fielder_id",
    "fielder_name",
    "fielder_omniscience",
    "fielder_omniscience_mul_vibe",
    "fielder_multiplier",
    "fielder_vibes",
    "fielder_mods",
    "ballpark_grandiosity",
    "ballpark_fortification",
    "ballpark_obtuseness",
    "ballpark_ominousness",
    "ballpark_inconvenience",
    "ballpark_viscosity",
    "ballpark_forwardness",
    "ballpark_mysticism",
    "ballpark_elongation",
    "ballpark_filthiness",
    "game_id",
    "stadium_id",
    "play_count",
    "weather",
    "ball_count",
    "strike_count",
    "out_count",
    "season",
    "day",
    "top_of_inning",
    "home_score",
    "away_score",
    "inning",
    "batting_team_roster_size",
    "pitching_team_roster_size",
    "baserunner_count",
]

In [7]:
for season in [11, 12, 13, 14]:
    df_s = df.loc[df['season'] == season, out_cols]
    df_s.to_csv(f"../roll_data/out-s{season+1}.csv")

In [8]:
def apply_doubles_threshold(row):
    if row["season"] in [11, 12]:
        # S12 and S13: perhaps slightly fiddly ballpark coefs, but no outliers.
        # ballpark coefs sum to 0.6, so there's that
        threshold = (
            0.17
            + 0.200 * row["batter_musclitude_mul_vibe"]
            - 0.040 * row["pitcher_overpowerment_mul_vibe"]
            - 0.010 * row["fielder_chasiness_mul_vibe"]
            + 0.027 * row["fwd_center"]
            - 0.015 * row["elong_center"]
            - 0.010 * row["omi_center"]
            - 0.008 * row["visc_center"]
        )
    elif row["season"] == 13:
        # S14: they nerfed chasiness slightly and lowered the intercept to compensate
        threshold = (
            0.165
            + 0.200 * row["batter_musclitude_mul_vibe"]
            - 0.040 * row["pitcher_overpowerment_mul_vibe"]
            - 0.009 * row["fielder_chasiness_mul_vibe"]
            + 0.027 * row["fwd_center"]
            - 0.015 * row["elong_center"]
            - 0.010 * row["omi_center"]
            - 0.008 * row["visc_center"]
        )
    elif row["season"] == 14:
        # S15: they nerfed chasiness slightly (again?) and lowered the intercept to compensate (again?)
        threshold = (
            0.16
            + 0.200 * row["batter_musclitude_mul_vibe"]
            - 0.040 * row["pitcher_overpowerment_mul_vibe"]
            - 0.008 * row["fielder_chasiness_mul_vibe"]
            + 0.027 * row["fwd_center"]
            - 0.015 * row["elong_center"]
            - 0.010 * row["omi_center"]
            - 0.008 * row["visc_center"]
        )
    return threshold


In [9]:
def apply_triples_threshold(row):
    if row["season"] in [11, 12]:
        # S12 and S13: not that bad...
        # multipliers work, but affinity for crows doesn't seem to affect fielders
        # grand and obt have the same coefficient, omi and visc have the same coefficient
        threshold = (
            0.05
            + 0.200 * row["batter_ground_friction_mul_vibe"]
            - 0.040 * row["pitcher_overpowerment_mul_vibe"]
            - 0.060 * row["fielder_chasiness_mul_vibe"]
            + 0.020 * row["fwd_center"]
            + 0.035 * row["grand_center"]
            + 0.035 * row["obt_center"]
            - 0.005 * row["omi_center"]
            - 0.005 * row["visc_center"]
        )
    elif row["season"] in [13, 14]:
        # S14-15: slightly fiddly but it works so...
        # multipliers work, but affinity for crows doesn't seem to affect fielders
        # grand and obt have the same coefficient, omi and visc have the same coefficient
        threshold = (
            0.045
            + 0.2000 * row["batter_ground_friction_mul_vibe"]
            - 0.0400 * row["pitcher_overpowerment_mul_vibe"]
            - 0.0500 * row["fielder_chasiness_mul_vibe"]
            + 0.0200 * row["fwd_center"]
            + 0.0340 * row["grand_center"]
            + 0.0340 * row["obt_center"]
            - 0.0065 * row["omi_center"]
            - 0.0065 * row["visc_center"]
        )
    return threshold


In [10]:
all_files = glob.glob("../roll_data/*-doubles.csv")

df = pd.concat((pd.read_csv(f, dtype=column_dtypes) for f in all_files), ignore_index=True)

# seasons = [11, 12]
# seasons = [11, 12, 13, 14]
# seasons = [14]
# seasons = [13]
# df = df[df['season'].isin(seasons)]
len(df)


46135

In [11]:
# zero out scattered vibes
df.loc[df["pitcher_mods"].astype(str).str.contains("SCATTERED"), "pitcher_vibes"] = 0
df.loc[df["batter_mods"].astype(str).str.contains("SCATTERED"), "batter_vibes"] = 0
df.loc[df["fielder_mods"].astype(str).str.contains("SCATTERED"), "fielder_vibes"] = 0

for attr in [
    "batter_buoyancy",
    "batter_divinity",
    "batter_martyrdom",
    "batter_moxie",
    "batter_musclitude",
    "batter_patheticism",
    "batter_thwackability",
    "batter_tragicness",
    "batter_coldness",
    "batter_overpowerment",
    "batter_ruthlessness",
    "batter_shakespearianism",
    "batter_suppression",
    "batter_unthwackability",
    "batter_base_thirst",
    "batter_continuation",
    "batter_ground_friction",
    "batter_indulgence",
    "batter_laserlikeness",
    "batter_anticapitalism",
    "batter_chasiness",
    "batter_omniscience",
    "batter_tenaciousness",
    "batter_watchfulness",
    "batter_pressurization",
    "batter_cinnamon",
]:
    df[attr + "_mul_vibe"] = df[attr] * df["batter_multiplier"] * (1 + 0.2 * df["batter_vibes"])

for attr in [
    "pitcher_buoyancy",
    "pitcher_divinity",
    "pitcher_martyrdom",
    "pitcher_moxie",
    "pitcher_musclitude",
    "pitcher_patheticism",
    "pitcher_thwackability",
    "pitcher_tragicness",
    "pitcher_ruthlessness",
    "pitcher_overpowerment",
    "pitcher_unthwackability",
    "pitcher_shakespearianism",
    "pitcher_suppression",
    "pitcher_coldness",
    "pitcher_base_thirst",
    "pitcher_continuation",
    "pitcher_ground_friction",
    "pitcher_indulgence",
    "pitcher_laserlikeness",
    "pitcher_anticapitalism",
    "pitcher_chasiness",
    "pitcher_omniscience",
    "pitcher_tenaciousness",
    "pitcher_watchfulness",
    "pitcher_pressurization",
    "pitcher_cinnamon",
]:
    df[attr + "_mul_vibe"] = df[attr] * df["pitcher_multiplier"] * (1 + 0.2 * df["pitcher_vibes"])

if "fielder_vibes" in df:
    for attr in [
        "fielder_anticapitalism",
        "fielder_chasiness",
        "fielder_omniscience",
        "fielder_tenaciousness",
        "fielder_watchfulness",
    ]:
        df[attr + "_mul_vibe"] = df[attr] * df["fielder_multiplier"] * (1 + 0.2 * df["fielder_vibes"])

df["grand_center"] = df["ballpark_grandiosity"] - 0.5
df["fort_center"]  = df["ballpark_fortification"] - 0.5
df["obt_center"]   = df["ballpark_obtuseness"] - 0.5
df["omi_center"]   = df["ballpark_ominousness"] - 0.5
df["fwd_center"]   = df["ballpark_forwardness"] - 0.5
df["visc_center"]  = df["ballpark_viscosity"] - 0.5
df["incon_center"] = df["ballpark_inconvenience"] - 0.5
df["elong_center"] = df["ballpark_elongation"] - 0.5

df = df.copy()

In [12]:
out_cols = [
    "event_type",
    "roll",
    "passed",
    "batter_id",
    "batter_name",
    "batter_musclitude",
    "batter_musclitude_mul_vibe",
    "batter_multiplier",
    "batter_vibes",
    "batter_mods",
    "batting_team_mods",
    "pitcher_id",
    "pitcher_name",
    "pitcher_overpowerment",
    "pitcher_overpowerment_mul_vibe",
    "pitcher_multiplier",
    "pitcher_vibes",
    "pitcher_mods",
    "pitching_team_mods",
    "fielder_id",
    "fielder_name",
    "fielder_chasiness",
    "fielder_chasiness_mul_vibe",
    "fielder_multiplier",
    "fielder_vibes",
    "fielder_mods",
    "grand_center",
    "fort_center",
    "obt_center",
    "omi_center",
    "fwd_center",
    "visc_center",
    "incon_center",
    "elong_center",
    "game_id",
    "stadium_id",
    "play_count",
    "weather",
    "ball_count",
    "strike_count",
    "out_count",
    "season",
    "day",
    "top_of_inning",
    "home_score",
    "away_score",
    "inning",
    "batting_team_roster_size",
    "pitching_team_roster_size",
    "baserunner_count",
]

In [13]:
for season in [11, 12, 13, 14]:
    df_s = df.loc[df['season'] == season, out_cols]
    df_s.to_csv(f"../roll_data/doubles-s{season+1}.csv")

In [14]:
all_files = glob.glob("../roll_data/*-triples.csv")

df = pd.concat((pd.read_csv(f, dtype=column_dtypes) for f in all_files), ignore_index=True)

# seasons = [11, 12]
# seasons = [11, 12, 13, 14]
# seasons = [14]
# seasons = [13]
# df = df[df['season'].isin(seasons)]
len(df)

52489

In [15]:
# zero out scattered vibes
df.loc[df["pitcher_mods"].astype(str).str.contains("SCATTERED"), "pitcher_vibes"] = 0
df.loc[df["batter_mods"].astype(str).str.contains("SCATTERED"), "batter_vibes"] = 0
df.loc[df["fielder_mods"].astype(str).str.contains("SCATTERED"), "fielder_vibes"] = 0

for attr in [
    "batter_buoyancy",
    "batter_divinity",
    "batter_martyrdom",
    "batter_moxie",
    "batter_musclitude",
    "batter_patheticism",
    "batter_thwackability",
    "batter_tragicness",
    "batter_coldness",
    "batter_overpowerment",
    "batter_ruthlessness",
    "batter_shakespearianism",
    "batter_suppression",
    "batter_unthwackability",
    "batter_base_thirst",
    "batter_continuation",
    "batter_ground_friction",
    "batter_indulgence",
    "batter_laserlikeness",
    "batter_anticapitalism",
    "batter_chasiness",
    "batter_omniscience",
    "batter_tenaciousness",
    "batter_watchfulness",
    "batter_pressurization",
    "batter_cinnamon",
]:
    df[attr + "_mul_vibe"] = df[attr] * df["batter_multiplier"] * (1 + 0.2 * df["batter_vibes"])

for attr in [
    "pitcher_buoyancy",
    "pitcher_divinity",
    "pitcher_martyrdom",
    "pitcher_moxie",
    "pitcher_musclitude",
    "pitcher_patheticism",
    "pitcher_thwackability",
    "pitcher_tragicness",
    "pitcher_ruthlessness",
    "pitcher_overpowerment",
    "pitcher_unthwackability",
    "pitcher_shakespearianism",
    "pitcher_suppression",
    "pitcher_coldness",
    "pitcher_base_thirst",
    "pitcher_continuation",
    "pitcher_ground_friction",
    "pitcher_indulgence",
    "pitcher_laserlikeness",
    "pitcher_anticapitalism",
    "pitcher_chasiness",
    "pitcher_omniscience",
    "pitcher_tenaciousness",
    "pitcher_watchfulness",
    "pitcher_pressurization",
    "pitcher_cinnamon",
]:
    df[attr + "_mul_vibe"] = df[attr] * df["pitcher_multiplier"] * (1 + 0.2 * df["pitcher_vibes"])

if "fielder_vibes" in df:
    for attr in [
        "fielder_anticapitalism",
        "fielder_chasiness",
        "fielder_omniscience",
        "fielder_tenaciousness",
        "fielder_watchfulness",
    ]:
        df[attr + "_mul_vibe"] = df[attr] * df["fielder_multiplier"] * (1 + 0.2 * df["fielder_vibes"])

df["grand_center"] = df["ballpark_grandiosity"] - 0.5
df["fort_center"]  = df["ballpark_fortification"] - 0.5
df["obt_center"]   = df["ballpark_obtuseness"] - 0.5
df["omi_center"]   = df["ballpark_ominousness"] - 0.5
df["fwd_center"]   = df["ballpark_forwardness"] - 0.5
df["visc_center"]  = df["ballpark_viscosity"] - 0.5
df["incon_center"] = df["ballpark_inconvenience"] - 0.5
df["elong_center"] = df["ballpark_elongation"] - 0.5

df = df.copy()

In [16]:
out_cols = [
    "event_type",
    "roll",
    "passed",
    "batter_id",
    "batter_name",
    "batter_ground_friction",
    "batter_ground_friction_mul_vibe",
    "batter_multiplier",
    "batter_vibes",
    "batter_mods",
    "batting_team_mods",
    "pitcher_id",
    "pitcher_name",
    "pitcher_overpowerment",
    "pitcher_overpowerment_mul_vibe",
    "pitcher_multiplier",
    "pitcher_vibes",
    "pitcher_mods",
    "pitching_team_mods",
    "fielder_id",
    "fielder_name",
    "fielder_chasiness",
    "fielder_chasiness_mul_vibe",
    "fielder_multiplier",
    "fielder_vibes",
    "fielder_mods",
    "grand_center",
    "fort_center",
    "obt_center",
    "omi_center",
    "fwd_center",
    "visc_center",
    "incon_center",
    "elong_center",
    "game_id",
    "stadium_id",
    "play_count",
    "weather",
    "ball_count",
    "strike_count",
    "out_count",
    "season",
    "day",
    "top_of_inning",
    "home_score",
    "away_score",
    "inning",
    "batting_team_roster_size",
    "pitching_team_roster_size",
    "baserunner_count",
]

In [17]:
for season in [11, 12, 13, 14]:
    df_s = df.loc[df['season'] == season, out_cols]
    df_s.to_csv(f"../roll_data/triples-s{season+1}.csv")